# トピックモデル

テキストの分析、あるいはテキストマイニングの代表的な分析手法に **トピックモデル** があります。トピックは、テーマあるいは主題とも表現されます。
新聞を例に説明しましょう。もっとも、今どきは新聞を購読していない人も多いと思いますが、新聞を読んだことが一度もないという人まだ少数派でしょう。さて、新聞の記事には、経済、政治、科学、娯楽、スポーツなどの種類があります。これがトピックに相当します。また、あるトピックには出現しやすい単語というのがあります。
政治に関する記事であれば「国会」や「選挙」といった単語が多く出現することになりますし、エンターテインメントであれば「タレント」とか「ドラマ」といった単語が目に付くでしょう。ただし、「タレント」という単語が政治ジャンルには出現しないということはありません。タレントから政治家に転身する人は多いのですから、そうした記事では政治とエンターテインメントそれぞれに顕著な単語の両方が出現していると考えられます。


## LDA

**LDA** は確率モデルです。確率とは、単語が出現する確率になります。テキストごとに単語が出現する確率を推定します。すると、一部のテキスト集合でだけ高い確率で出現する単語の集合が明らかになることがあります。高確率で出現する単語の集合がトピックを構成するとみなすわけです。この単語の集合に政治関連の言葉が多数含まれていれば、その単語集合は「政治トピック」を構成すると解釈できることになります。

In [2]:
## https://github.com/yuukimiyo/GeneralPolicySpeechOfPrimeMinisterOfJapan
## からダウンロードしたファイルを解凍し、longfilenameフォルダにある
## utf8フォルダを data 直下に配置
import os
import re
files = ['data/utf8/' + path for path in os.listdir('data/utf8')]
## ファイル名から国会の番号と首相の名前を抽出
pattern = 'data/utf8/\\d{8}_(\\d{1,3}_[a-z]{1,}-[a-z]{1,})_general-policy-speech.txt'
results = [re.match(pattern, file_name) for file_name in files]
prime_names = [ res.group(1) for res in results]

In [3]:
stopwords = [],
with open('data/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = [w.strip() for w in f]
## ストップワードをさらに追加
stopwords.extend(["あの", "この", "ある", "する", "いる", "できる", "なる", "れる", "の", "は", "〇", "ソ", "もつ", "わが国", "われわれ","私たち","そのため","行なう","おこなう"])
## セットに変更（形態素が重複して登録されているのを避けるため）
stopwords = set(stopwords)
## ストップワードの要素数を確認
print(len(stopwords))

330


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
## 形態素解析器としてMeCabを指定
import my_mecab_stopwords as my_tokenizer
args={'stopwords_list': stopwords}
## フォルダからファイルを読み込んで辞書を作成
vectorizer = CountVectorizer(input='filename', lowercase=False,
                             max_df=0.5, max_features=200,
                             tokenizer=lambda text: my_tokenizer.tokens(text, **args))
## 文書単語行列を生成
prime_dtm = vectorizer.fit_transform(files)


LDA を適用すべき単語文書行列が生成されました。

In [6]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(
    ## トピック数
    n_components=5,
    ## 推定における計算回数
    max_iter=20,
    ## 乱数の種を指定
    random_state = 123
)
topic_data = lda.fit_transform(prime_dtm)

トピックが推定されました。まず、それぞれのトピックで頻出する語を確認してみましょう。

In [7]:
## トピックごとに高頻度で現れる単語の一覧
features = vectorizer.get_feature_names()
## トピック数として5を指定した
for tn in range(5):
    print("topic number: " + str(tn))
    ## トピックごとに上位20語を表示
    row = lda.components_[tn]
    words = ', '.join([features[i] for i in row.argsort()[:-20-1:-1]])
    print(words, "\n")

topic number: 0
皆さん, 被災地, 再生, 危機, 復興, 未来, 取り戻す, エネルギー, 日本人, 支える, 日本経済, 歴史, もたらす, 行動, 安心, 進む, 投資, 世代, 力強い, 現実 

topic number: 1
政治改革, 役割, 国連, 皆様, 構築, 民主主義, 歴史, 秩序, 展開, サミット, 理念, 公正, 是正, 税制, 事態, 不可欠, 向かう, 構造, 世界経済, 土地 

topic number: 2
物価, はかる, 所存, 存じる, 沖, 繩, 貿易, 収支, 消費者, 長期, 諸君, 石油, 需要, 輸入, 上昇, 事態, 均衡, 次第, 確信, 講ずる 

topic number: 3
皆様, 所存, 次第, 世界経済, 昭和, 行政改革, 石油, 調和, サミット, エネルギー, 国土, 税制, 配慮, 割り, 展望, 首脳, 国際的, 協力関係, 民間, 速やか 

topic number: 4
皆様, 議論, 安心, 構造改革, 構築, 取り組み, システム, 民間, 沖縄, 医療, つくる, 役割, 含める, 年金, 平成, 支える, 超える, 活動, 再生, 合意 



各トピックにおいて出現確率の高い単語を上位から20語だけ抽出してみました。また、トピックモデルでは、それぞれの所信表明演説において、5つのトピックが占める割合も推定されています。確認のため10の演説に絞って、トピックの割合を確認してみます。


In [9]:
for i,lda in enumerate(topic_data[:10]):
    topicid=[j for j, x in enumerate(lda) if x == max(lda)]
    print('speech = ' + prime_names [i]  + ' : estimated = ' + str(lda)  + ' : max topic = ' + str(topicid[0]))

speech = 47_sato-eisaku : estimated = [0.0011297  0.00113668 0.87093112 0.12567359 0.00112892] : max topic = 2
speech = 185_abe-shinzo : estimated = [0.58700919 0.00163806 0.04555288 0.00164127 0.3641586 ] : max topic = 0
speech = 26_kishi-nobusuke : estimated = [0.01126438 0.30459626 0.6613177  0.01145677 0.01136488] : max topic = 2
speech = 163_koizumi-jyunichiro : estimated = [0.1368198  0.00266661 0.00270548 0.00269862 0.85510949] : max topic = 4
speech = 49_sato-eisaku : estimated = [0.00174919 0.00175309 0.99298649 0.00175368 0.00175755] : max topic = 2
speech = 25_hatoyama-ichiro : estimated = [0.00471127 0.00486103 0.98087367 0.00477248 0.00478155] : max topic = 2
speech = 70_tanaka-kakuei : estimated = [0.02930144 0.00160042 0.85146087 0.00159523 0.11604203] : max topic = 2
speech = 53_sato-eisaku : estimated = [0.00170945 0.00172058 0.99313085 0.00172645 0.00171267] : max topic = 2
speech = 90_ohira-masayoshi : estimated = [0.00199984 0.00201557 0.29118928 0.70277272 0.002022

In [10]:
import pandas as pd
df = pd.DataFrame(columns=['speech', 'topic', 'ratio'])
for i,lda in enumerate(topic_data):
    topicid=[j for j, x in enumerate(lda) if x == max(lda)]
    df = df.append({'speech': prime_names [i], 'topic': topicid[0], 'ratio': max(lda)}, ignore_index=True)

作成されたデータフレームの冒頭を確認しましょう。

In [18]:
df.head()

,speech,topic,ratio
0,47_sato-eisaku,2,0.870931
1,185_abe-shinzo,0,0.587009
2,26_kishi-nobusuke,2,0.661318
3,163_koizumi-jyunichiro,4,0.855109
4,49_sato-eisaku,2,0.992986


佐藤栄作氏の第47回国会での演説ではトピック3番（添字で`[2]`となります）が約87パーセントを占めることがわかります。一方、安倍晋三氏の第185回国会での演説では約59パーセントが1番目のトピックで占められています。
以下、トピックの分類結果から、5種類のトピックのいずれが高い割合で出現しているかを調べ、そのパーセンテージが高い順に5個程度抜き出してみましょう。

In [11]:
<for i in range(5):
    print(df.query('topic == @i').sort_values(['ratio', 'speech'], ascending=False).head(5))

                speech topic     ratio
13      183_abe-shinzo     0  0.993022
40  181_noda-yoshihiko     0  0.709630
1       185_abe-shinzo     0  0.587009
42  179_noda-yoshihiko     0  0.531605
                   speech topic     ratio
78  127_hosokawa-morihiro     1  0.995411
33    122_miyazawa-kiichi     1  0.993726
74      121_kaifu-toshiki     1  0.927573
62    125_miyazawa-kiichi     1  0.888629
61  128_hosokawa-morihiro     1  0.834693
             speech topic     ratio
31  44_ikeda-hayato     2  0.996771
75  41_ikeda-hayato     2  0.993509
18   64_sato-eisaku     2  0.993249
47   66_sato-eisaku     2  0.993205
7    53_sato-eisaku     2  0.993131
                   speech topic     ratio
9   100_nakasone-yasuhiro     3  0.995703
76        85_fukuda-takeo     3  0.994598
21   97_nakasone-yasuhiro     3  0.994111
26        95_suzuki-zenko     3  0.964874
36  107_nakasone-yasuhiro     3  0.912326
                    speech topic     ratio
81        150_mori-yoshiro     4  0.997193

この結果を見ると、総理大臣に就任した時期が近い演説では、共通のトピックが大きな割合を占めているようです。これは、それぞれの時代特有の政治的・経済的課題があり、首相が代わっても、その課題は共有されていたと解釈できるのかもしれません。そもそも日本の戦後政治ではほぼ一貫して自民党が政権を担っているので、政党の違いによる政策の違いというのは所信表明演説においてはほとんど顕在化したことがなく、演説の趣旨の違いはほぼ時代的な課題によっているのかもしれません。
